In [0]:
# Importando libs
from pyspark.sql.functions import months_between, lit, to_date
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, sum as _sum, count, min as _min, max as _max, datediff
import pandas as pd
import datetime as dt
import numpy as np

#Importando bases
user_orders = spark.table('workspace.tabelas_ifood.user_orders')
consumer = spark.table('workspace.tabelas_ifood.consumer')
ab_ref = spark.table('workspace.tabelas_ifood.ab_test_ref')
restaurant = spark.table('workspace.tabelas_ifood.restaurant')

In [0]:
# Gerando variáveis descritivas para os usuários

user_orders_pd = user_orders.toPandas()
user_orders_pd['order_created_at'] = pd.to_datetime(user_orders_pd['order_created_at'])
user_orders_pd['merchant_zip_code'] = user_orders_pd['merchant_zip_code'].astype('str')

user_orders_summary = user_orders_pd.groupby(['customer_id']).agg(
    order_count=('order_id', 'count'),
    merchant_zip_code=('merchant_zip_code', 'first'),
    avg_order_amount=('order_total_amount', 'mean')
).reset_index().reset_index(drop=True)

active_days = user_orders_pd.groupby('customer_id')['order_created_at'].agg(['min', 'max']).reset_index().reset_index(drop=True)
active_days['active_days'] = (active_days['max'] - active_days['min']).dt.days
user_orders_summary = user_orders_summary.merge(active_days[['customer_id','active_days']], on='customer_id', how='left')
user_orders_summary['order_frequency'] = user_orders_summary['active_days']/user_orders_summary['order_count']

In [0]:
# Gerando métricas para cada cidade
cities_orders_summary = user_orders_pd[['merchant_zip_code','customer_id','order_id','order_total_amount']].copy()
cities_orders_summary = cities_orders_summary[cities_orders_summary['merchant_zip_code']!='0'].reset_index(drop='True')
cities_orders_summary = cities_orders_summary.groupby(['customer_id','merchant_zip_code']).agg(
    order_count=('order_id', 'count'),
    avg_order_amount=('order_total_amount', 'mean')
).reset_index().reset_index(drop=True)

In [0]:
city_active_days = user_orders_pd.groupby('merchant_zip_code')['order_created_at'].agg(['min', 'max']).reset_index().reset_index(drop=True)
city_active_days['active_days'] = (active_days['max'] - active_days['min']).dt.days
cities_orders_summary = cities_orders_summary.merge(city_active_days[['merchant_zip_code','active_days']], on='merchant_zip_code', how='left')
cities_orders_summary['city_order_frequency'] = cities_orders_summary['active_days']/cities_orders_summary['order_count']

In [0]:
# Criando função relativa para segmentação

def calculate_quartiles(df, column):
    Q1 = df[column].quantile(0.25)
    Q2 = df[column].quantile(0.50)
    Q3 = df[column].quantile(0.75)
    return Q1, Q2, Q3

In [0]:
frequency_quartiles_dict = {}
for zip_code in cities_orders_summary['merchant_zip_code'].unique():
    zip_code_df = cities_orders_summary[cities_orders_summary['merchant_zip_code'] == zip_code]
    Q1, Q2, Q3 = calculate_quartiles(zip_code_df, 'city_order_frequency')
    frequency_quartiles_dict[zip_code] = {'Q1': Q1, 'Q2': Q2, 'Q3': Q3}

In [0]:
amount_quartiles_dict = {}
for zip_code in cities_orders_summary['merchant_zip_code'].unique():
    zip_code_df = cities_orders_summary[cities_orders_summary['merchant_zip_code'] == zip_code]
    Q1, Q2, Q3 = calculate_quartiles(zip_code_df, 'avg_order_amount')
    amount_quartiles_dict[zip_code] = {'Q1': Q1, 'Q2': Q2, 'Q3': Q3}

In [0]:
# Segmentando usuários conforme o zip_code

def get_price_range(zip_code, avg_order_amount):
    quartiles = amount_quartiles_dict.get(zip_code, {})
    Q1 = quartiles.get('Q1', float('inf'))
    Q2 = quartiles.get('Q2', float('inf'))
    Q3 = quartiles.get('Q3', float('inf'))
    
    if avg_order_amount < Q1:
        return '$'
    elif Q1 <= avg_order_amount < Q2:
        return '$$'
    elif Q2 <= avg_order_amount < Q3:
        return '$$$'
    else:
        return '$$$$'

user_orders_summary['price_range'] = user_orders_summary.apply(
    lambda row: get_price_range(row['merchant_zip_code'], row['avg_order_amount']), axis=1
)

In [0]:
def get_order_frequency(zip_code, order_frequency):
    quartiles = frequency_quartiles_dict.get(zip_code, {})
    Q1 = quartiles.get('Q1', float('inf'))
    Q2 = quartiles.get('Q2', float('inf'))
    Q3 = quartiles.get('Q3', float('inf'))
    
    if order_frequency < Q1:
        return 'muito alta'
    elif Q1 <= order_frequency < Q2:
        return 'alta'
    elif Q2 <= order_frequency < Q3:
        return 'moderada'
    else:
        return 'baixa'

user_orders_summary['frequency_range'] = user_orders_summary.apply(
    lambda row: get_order_frequency(row['merchant_zip_code'], row['order_frequency']), axis=1
)

In [0]:
user_labeled = user_orders_summary[['customer_id','merchant_zip_code','price_range','frequency_range']].copy()
user_labeled_spark = spark.createDataFrame(user_labeled)

In [0]:
user_labeled_spark.write.mode('overwrite').saveAsTable('workspace.tabelas_ifood.user_labels')